In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import sys
import warnings
import numpy as np
from scipy.signal import filtfilt
from scipy.signal import firwin
from scipy.signal import resample
warnings.filterwarnings('ignore')

DATA_PATH = '/workspaces/Tracking-Gazes-on-Museum-Pieces-Data-Plus/data'

def concat_df(DATA_PATH):
    df = pd.DataFrame()
    for folder in os.listdir(DATA_PATH):
        if not os.path.isdir(os.path.join(DATA_PATH, folder)):
            continue
        temp_path = os.path.join(DATA_PATH, folder, 'gaze.csv')
        temp_df = pd.read_csv(temp_path)
        temp_df['folder_id'] = folder
        df = pd.concat([df, temp_df])
    return df

"""
print('Participant count', gaze_df['section id'].nunique())
gaze_df = pd.merge(demographic_data, gaze_df, left_on='codice_eyetr_museo', right_on='folder_id', how='right')
gaze_df.replace({'m': 0, 'f': 1}, inplace=True)
print('Row count', gaze_df.shape[0])
print('Participant count', gaze_df['section id'].nunique())
print('Participant count', gaze_df['codice_eyetr_museo'].nunique())
display(gaze_df.head())
"""

demographic_data = pd.read_excel(os.path.join(DATA_PATH, 'demographic.xlsx'))
orig_data = concat_df(DATA_PATH)
modified_data = pd.read_csv(os.path.join(DATA_PATH, 'all_gaze.csv'), compression='gzip')
print(orig_data.columns)
print(modified_data.columns)
# keep_columns = ['timestamp [ns]_for_grouping', 'participant_folder', 'ref_center_x', 'ref_center_y']
# modified_data = modified_data[keep_columns]

Index(['section id', 'recording id', 'timestamp [ns]', 'gaze x [px]',
       'gaze y [px]', 'worn', 'fixation id', 'blink id', 'folder_id'],
      dtype='object')
Index(['index', 'section id', 'recording id', 'timestamp [ns]', 'gaze x [px]',
       'gaze y [px]', 'worn', 'fixation id', 'blink id',
       'timestamp [ns]_for_grouping', 'ref_center_x', 'ref_center_y',
       'ref_coordinates', 'tag', 'ts', 'increment_marker', 'seconds_id',
       'participant_folder', 'art_piece', 'participant_id'],
      dtype='object')


In [9]:
def cluster_fix(eyedat):
    samprate = 5/1000  # in secs e.g. 200 Hz -> 5 ms/sample -> 5/1000
    variables = ['Dist', 'Vel', 'Accel', 'Angular Velocity']  # parameters to be extracted



    for cndlop in range(len(eyedat)):
        if eyedat[cndlop].shape[1] > 500/samprate/1000:
            # --- Filtering Extract Parameters from Eye Traces --- #
            x = eyedat[cndlop][0, :]  # *24+400;  # converts dva to pixel and data from [-400,400] to [0,800]
            y = eyedat[cndlop][1, :]  # *24+300;  # converts dva to pixel and from [-300,300] to [0,600]
            x = np.concatenate((x[buffer::-1], x, x[-1:-buffer-1:-1]))  # add buffer for filtering
            y = np.concatenate((y[buffer::-1], y, y[-1:-buffer-1:-1]))  # add buffer for filtering
            x = resample(x, int(samprate*1000), 1)  # up sample to 1000 Hz
            y = resample(y, int(samprate*1000), 1)  # up sample to 1000 Hz
            xss = filtfilt(flt, 1, x)
            yss = filtfilt(flt, 1, y)
            xss = xss[100:-100]  # remove buffer after filtering
            yss = yss[100:-100]  # remove buffer after filtering
            x = x[100:-100]  # remove buffer after filtering
            y = y[100:-100]  # remove buffer after filtering
            velx = np.diff(xss)
            vely = np.diff(yss)
            vel = np.sqrt(velx**2 + vely**2)  # velocity
            accel = np.abs(np.diff(vel))  # acceleration
            angle = 180*np.arctan2(vely, velx)/np.pi
            vel = vel[:-1]
            rot = np.zeros(len(xss)-2)  # angular velocity
            dist = np.zeros(len(xss)-2)  # distance
            for a in range(len(xss)-2):
                rot[a] = np.abs(angle[a] - angle[a+1])
                dist[a] = np.sqrt((xss[a] - xss[a+2])**2 + (yss[a] - yss[a+2])**2)

            rot[rot > 180] = rot[rot > 180] - 180
            rot = 360 - rot  # want angular velocity to be small so fixation values are all small

            points = np.column_stack((dist, vel, accel, rot))
            for ii in range(points.shape[1]):
                thresh = np.mean(points[:, ii]) + 3 * np.std(points[:, ii])  # move outliers
                points[points[:, ii] > thresh, ii] = thresh
                points[:, ii] = points[:, ii] - np.min(points[:, ii])
                points[:, ii] = points[:, ii] / np.max(points[:, ii])

    return eyedat        

modified_data = modified_data.groupby('participant_folder').apply(cluster_fix).reset_index(drop=True)

ValueError: Invalid cutoff frequency: frequencies must be greater than 0 and less than fs/2.